<a href="https://colab.research.google.com/github/nicoloceneda/Python-edu/blob/master/TensorFlow_Custom_Layers_with_Keras_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow - Custom Layer with Keras API 

---



In [0]:
import tensorflow as tf
from tensorflow.keras import layers

## Adding weights
A Layer is the main data structure that encapsulates a state (the weights w and b) and a transformation from inputs to outputs (the forward pass defined in call). 

**Add trainable weights** to a layer `manually` or using `add_weight`.

In [3]:
# Add trainable weights manually
class Linear(layers.Layer):

  def __init__(self, units=32, input_dim=32):
    super(Linear, self).__init__()
    w_init = tf.random_normal_initializer()
    self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units), dtype='float32'), trainable=True)
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(initial_value=b_init(shape=(units, ), dtype='float32'), trainable=True)

  def call(self, inputs):
    return tf.linalg.matmul(inputs, self.w) + self.b

x = tf.ones((2, 2))

linear_layer = Linear(units=4, input_dim=2)
y = linear_layer(inputs=x)
print(y.numpy())

[[0.01778614 0.00810714 0.11528654 0.02348447]
 [0.01778614 0.00810714 0.11528654 0.02348447]]


In [6]:
# Add trainable weights using add_weight
class Linear(layers.Layer):

  def __init__(self, units=32, input_dim=32):
    super(Linear, self).__init__()
    self.w = self.add_weight(shape=(input_dim, units), initializer='random_normal', trainable=True)
    self.b = self.add_weight(shape=(units, ), initializer='zeros', trainable=True)

  def call(self, inputs):
    return tf.linalg.matmul(inputs, self.w) + self.b

  x = tf.ones((2, 2))

  linear_layer = Linear(units=4, input_dim=2)
  y = linear_layer(inputs=x)
  print(y.numpy())

[[ 0.02587924  0.14166737 -0.03810942 -0.06730217]
 [ 0.02587924  0.14166737 -0.03810942 -0.06730217]]


**Add non-trainable weights** (which are meant not to be taken into account during the backpropagation process of training) to a Layer `manually`. 

In [9]:
class ComputeSum(layers.Layer):

  def __init__(self, input_dim=32):
    super(ComputeSum, self).__init__()
    self.total = tf.Variable(initial_value=tf.zeros((input_dim, )), trainable=False)

  def call(self, inputs):
    self.total.assign_add(tf.reduce_sum(inputs, axis=0))
    return self.total

x = tf.ones((2, 2))

my_sum = ComputeSum(input_dim=2)
y = my_sum(inputs=x)
print(y.numpy())
y = my_sum(inputs=x)
print(y.numpy())

[2. 2.]
[4. 4.]


**Defer weight creation** until the shape of the inputs is known using `add_weight` inside `build(input_shape)`. Doing so, the weights are created dynamically the first time the layer is called. This is the best practice as in many cases the size of the inputs is not known in advance.

In [9]:
class Linear(layers.Layer):

  def __init__(self, units=32):
    super(Linear, self).__init__()
    self.units = units

  def build(self, input_dim=32):
    self.w = self.add_weight(shape=(input_dim[-1], self.units), initializer='random_normal', trainable=True)
    self.b = self.add_weight(shape=(self.units, ), initializer='zeros', trainable=True)
  
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

x = tf.ones((2, 2))

linear_layer = Linear(units=4)
y = linear_layer(inputs=x)
print(y.numpy())

(2, 3) 3
[[ 0.10048345  0.10018738 -0.03829457 -0.0474291 ]
 [ 0.10048345  0.10018738 -0.03829457 -0.0474291 ]]


## Recursively defined layers and losses
Define **layers recursively** (a layer instance is assigned as attribute of another layer) by defining them in the `__init__` method.

In [11]:
class Linear(layers.Layer):

  def __init__(self, units=32):
    super(Linear, self).__init__()
    self.units = units

  def build(self, input_shape):
    self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True)
    self.b = self.add_weight(shape=(self.units, ), initializer='zeros', trainable=True)

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

class MLPBlock(layers.Layer):

  def __init__(self):
    super(MLPBlock, self).__init__()
    self.linear_1 = Linear(units=32)
    self.linear_2 = Linear(units=32)
    self.linear_3 = Linear(units=1)

  def call(self, inputs):
    x = self.linear_1(inputs)
    x = tf.nn.relu(x)
    x = self.linear_2(x)
    x = tf.nn.relu(x)
    return self.linear_3(x)

x = tf.ones(shape=(3, 64))

mlp = MLPBlock()
y = mlp(inputs=x)
print(y.numpy())

[[-0.01583333]
 [-0.01583333]
 [-0.01583333]]
weights: 6
trainable weights: 6
